# Code to plot the TTV diagram (Residual from the linear fit vs epoch)

Importing all the necessary packages. In this project, we basically used glob, pandas and bokeh. However, we didn't import the whole package. Instead we imported only the function that we need.

In [4]:
from glob import glob
from pandas import read_fwf
from bokeh.plotting import save, figure, output_file, show
from bokeh.models import ColumnDataSource, OpenURL, TapTool, BasicTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.transform import factor_cmap
from bokeh.io import export_png

First, we fetch all the data files that is in "modified_data" folder. In the modified data folder, we added one more column to the original data file. This column indictes a light curve name for each 'ThisStudy' and just a random garbage value for others.

In [5]:
file_list = glob('data/mod_data/*.txt')

Then for each TTV data file, we plotted the linear residual (min) vs epoch (cycle). Here, we also added a hover effect and link to the light curve for 'ThisStudy' LCType. Follow the code comment for details.

In [6]:
for data_file in file_list:        
    
    output = "plots/ttv/" + data_file[14:-4] + ".html"
    output_file(output)

    end = data_file.find('_oc')
    if data_file[14:18] == "hatp":
        name = "HAT-P-" + data_file[18:end]
    elif data_file[14:18] == "wasp":
        name = "WASP-" + data_file[18:end]

    ref = read_fwf(data_file, skiprows=1, header=None, sep="\t+")
    ref = ref.drop(ref[ref[21] == 1].index)

    df = ref.drop(ref[ref[8] == 'ThisStudy'].index)
    this = ref.drop(ref[ref[8] != 'ThisStudy'].index)

    source = ColumnDataSource(data=dict(
                epoch = df[4],
                res = df[6]*1440,
                LCType = df[8], 
                err_l = (df[6] - df[1])*1440,
                err_h = (df[6] + df[1])*1440,
        
                img_link = "../../" + df[22] + "preview.png",
                link = "../../" + df[22] + "preview.html"
            ))

    sourceThis = ColumnDataSource(data=dict(
                epoch=this[4],
                res=this[6]*1440,
                LCType=this[8], 
                err_l = (this[6] - this[1])*1440,
                err_h = (this[6] + this[1])*1440,
    
                img_link = "../../" + this[22] + "lc.png",
                link = "../../" + this[23] + "preview.html"
            ))

    p = figure(sizing_mode="stretch_both",                             
        tools="pan, box_zoom, save, reset, wheel_zoom, help", 
        title="TTV Diagram for " + name,
        x_axis_label = "Epoch (cycle)", 
        y_axis_label = "O-C (min)")

    p.segment(x0=df[4],
        y0=(df[6] - df[1])*1440,
        x1=df[4],
        y1=(df[6] + df[1])*1440,
        color = 'black'
        )

    p.segment(x0=df[4],
        y0=(df[6] - df[1])*1440,
        x1=df[4],
        y1=(df[6] + df[1])*1440,
        color = 'black'
        )

    p.circle('epoch', 'res',                                         
                size=10,                                              
                source=source,                                        
                color=factor_cmap('LCType',
                                ['gray', 'turquoise', 'darkblue'],
                                ['Database', 'Literature', 'TESS']),
                legend_group = 'LCType', 
                name = "needsHover"
            )

    p.circle('epoch', 'res',                     
                size=12,
                source=sourceThis,
                color=factor_cmap('LCType',
                                ['red'],
                                ['ThisStudy']),
                legend_group='LCType',
                name = "needsHover"
        )
    
    p.legend.title = "Light Curve Type"
    p.legend.location = "top_left" 

    hover = HoverTool(names=['needsHover'])
    hover.tooltips = """
              <div>
                <h3>{title} </h3>
                <div><strong>Click to see the details</strong></div>
                <div><img src="@img_link" alt="Sorry, the image couldn't load properly. If you see this message, please let the administrator know." width="200" /></div>
              </div>
            """.format(title = "Light Curve Preview")
    p.add_tools(hover)

    url = "@link"
    tap = TapTool(names = ["needsHover"]) 
    tap.callback = OpenURL(url=url)
    p.add_tools(tap)

    p.xaxis.axis_label_text_font_size = "15pt"
    p.yaxis.axis_label_text_font_size = "15pt"
    p.xaxis.major_label_text_font_size = "15pt"
    p.yaxis.major_label_text_font_size = "15pt"
    p.xaxis.formatter = BasicTickFormatter(use_scientific=False)
    p.yaxis.formatter = BasicTickFormatter(use_scientific=False)
    p.title.text_font_size = '20pt'
    p.legend.label_text_font_size = '13pt'
    p.legend.title_text_font_size = '14pt'

    #export_png(p, filename="plots/ttv_images/" + data_file[14:-4] + ".png") # exporting as png image to use in the homepage
    save(p)